In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [2]:
spark = SparkSession.builder.getOrCreate()

### Spark Sql fakeFriends.csv

In [6]:
def parseLine(line):
    line = line.split(",")
    return Row(id=int(line[0]), 
               name=str(line[1]), 
               age=int(line[2]), 
               friends=int(line[3]))

lines = spark.sparkContext.textFile("fakefriends.csv")
lines = lines.map(parseLine)

In [8]:
peopleSchema = spark.createDataFrame(lines).cache()
peopleSchema.createOrReplaceTempView("people")

teens = spark.sql("SELECT * FROM people WHERE age BETWEEN 13 AND 19")
for i, teen in enumerate(teens.collect()):    
    if i == 5 : break
    print(teen)

peopleSchema.groupBy("age").count().orderBy("age").show(5)

Row(id=21, name='Miles', age=19, friends=268)
Row(id=52, name='Beverly', age=19, friends=269)
Row(id=54, name='Brunt', age=19, friends=5)
Row(id=106, name='Beverly', age=18, friends=499)
Row(id=115, name='Dukat', age=18, friends=397)
+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
+---+-----+
only showing top 5 rows



#### Spark Sql  Dataframe

In [9]:
people = spark.read.option("header", "true").option("inferSchema", "true")\
        .csv("fakeFriendsHeader.csv")
    
people.printSchema()
people.filter(people["Name"].rlike("[\W-\W]")).show(5)
people.select(people["Name"], people["Age"]+10).show(5)

root
 |-- UserId: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Friends: integer (nullable = true)

+------+--------+---+-------+
|UserId|    Name|Age|Friends|
+------+--------+---+-------+
|     1|Jean-Luc| 26|      2|
|    13|Jean-Luc| 56|    444|
|    18|Jean-Luc| 45|    455|
|    34|Jean-Luc| 43|    249|
|    63|Jean-Luc| 58|     54|
+------+--------+---+-------+
only showing top 5 rows

+--------+----------+
|    Name|(Age + 10)|
+--------+----------+
|    Will|        43|
|Jean-Luc|        36|
|    Hugh|        65|
|  Deanna|        50|
|   Quark|        78|
+--------+----------+
only showing top 5 rows



### Friends By Age

In [23]:
people = spark.read.option("header", "true").option("inferSchema", "true")\
        .csv("fakeFriendsHeader.csv")
    
people.groupBy("Age").agg(functions.round(functions.avg("Friends"), 2).alias("Avg Friends"))\
.sort("Age").show(5)

+---+-----------+
|Age|Avg Friends|
+---+-----------+
| 18|     343.38|
| 19|     213.27|
| 20|      165.0|
| 21|     350.88|
| 22|     206.43|
+---+-----------+
only showing top 5 rows



### Word Count Better Sorted

#### !!! IMP
Dataframes work best with structured Data <br>
RDD would be better for this <br>
RDD can be converted to Dataframes <br>
Basically,<br>
**Load Data as an RDD then convert it to Dataframes for further proccessing**

In [76]:
bookDf = spark.read.text("Book")
wordDf = bookDf.select(functions.explode(functions.split(bookDf.value, r"\W+")).alias("word"))
'''
Explode function can be used to explode an Array of Array (nested Array) 
ArrayType(ArrayType(StringType)) columns to rows on PySpark DataFrame
'''
wordDf.show(5)

+----------+
|      word|
+----------+
|      Self|
|Employment|
|  Building|
|        an|
|  Internet|
+----------+
only showing top 5 rows



In [82]:
wordDf = wordDf.filter(wordDf.word != "")
wordDfLower = wordDf.select(functions.lower(wordDf.word).alias("Word"))
wordDfCount = wordDfLower.groupBy("Word").count()
wordDfCount.sort("count", ascending=False).show(5)

+----+-----+
|Word|count|
+----+-----+
| you| 1878|
|  to| 1828|
|your| 1420|
| the| 1292|
|   a| 1191|
+----+-----+
only showing top 5 rows



### Min Temperatures

In [112]:
customSchema = StructType([
    StructField("StationId", StringType(), nullable=True),
    StructField("Date", IntegerType(), nullable=True),
    StructField("MeasureType", StringType(), nullable=True),
    StructField("Temperature", FloatType(), nullable=True)
])

tempDf = spark.read.schema(customSchema).csv("1800.csv")
tempDf.printSchema()

root
 |-- StationId: string (nullable = true)
 |-- Date: integer (nullable = true)
 |-- MeasureType: string (nullable = true)
 |-- Temperature: float (nullable = true)



In [113]:
minTemp = tempDf.filter(tempDf.MeasureType == "TMIN")
stationTemp = minTemp.select("StationId", "Temperature")
minStationTemp = stationTemp.groupBy("StationId").min("Temperature")

In [114]:
minStationTemp = minStationTemp.withColumn("temperature",\
            functions.round(functions.col("min(temperature)") * 0.1 * (9.0 / 5.0) + 32.0, 2))\
            .select("stationID", "temperature").sort("temperature")
'''
function withColumn ---> To create a new column
'''

In [116]:
results = minStationTemp.collect()
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F


### Customer Orders

In [23]:
customerSchema = StructType([
    StructField("cust_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("amount_spent", FloatType()),
])
df = spark.read.schema(customerSchema).csv("customer-orders.csv")
df.printSchema()

root
 |-- cust_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- amount_spent: float (nullable = true)



In [24]:
df.groupBy("cust_id").agg(functions.round(functions.sum("amount_spent"), 2)\
                          .alias("amount_spent")).sort("amount_spent", ascending=False).show(5)

+-------+------------+
|cust_id|amount_spent|
+-------+------------+
|     68|     6375.45|
|     73|      6206.2|
|     39|     6193.11|
|     54|     6065.39|
|     71|     5995.66|
+-------+------------+
only showing top 5 rows

